# Exploration simulation of a Molten NaCl Salt with OpenMM

This notebook performs an NpT equilibration followed by an NVE production run.
The NVE simulation is intended for a first analysis of a system,
to get a basic idea of the time scales of this system,
which will be used to verify or fine-tune the simulation time and block size for the production run.

The Born-Huggins-Mayer-Tosi-Fumi force field for NaCl is implemented in `bhmtf.py`.
The MD implementation and some related utility functions can be found in `utils.py`.
This notebook merely ties these two together and performs some basic sanity checks on the results.

You can use this notebook as such, but it is also designed to be used as a step in a StepUp workflow
that performs all the MD simulations for a reasonably accurate computation of the conductivity of NaCl.
If set, the environment variable `REREP_NBARGS` is used to set the seed for the random part00ization of the thermostat and the velocities.

## Setup

In [ ]:
import mdtraj
import nglview
from bhmtf import add_nacl_forces, build_nacl_lattice
from openmm import unit
from openmm.app import PDBFile
from openmm.version import version as openmm_version
from stepup.core.api import amend
from utils import make_plots, runmd

print("OpenMM version:", openmm_version)

In [ ]:
# All parameters in this cell can be changed through papermill:
# - https://papermill.readthedocs.io/
# - https://reproducible-reporting.github.io/stepup-reprep/stable/reference/stepup.reprep.api/#stepup.reprep.api.execute_papermill

# Simulation parameters
seed = 42

# Physical parameters
temperature = 1100 * unit.kelvin
pressure = 1 * unit.bar

# The experimental density from https://doi.org/10.1021/jp5050332 is only used for initialization.
# Higher initial densities will let the system crystallize.
density = 1.4444 * unit.gram / unit.centimeter**3

# Time-related settings
timestep = 5 * unit.femtosecond
stride = 10
nstep_nvt = 4000
nstep_npt = 30000
nstep_nve = 8000
tau_thermostat = 1 * unit.picosecond

In [ ]:
# Inform StepUp of the output files it should expect, and which files are used as inputs.
prefix = f"sim{seed:04d}_part00"
print("Prefix:", prefix)
amend(
    inp=[
        "bhmtf.py",
        "utils.py",
    ],
    out=[
        f"output/{prefix}_first.pdb",
        f"output/{prefix}_nvt_traj.csv",
        f"output/{prefix}_nvt_traj.dcd",
        f"output/{prefix}_nvt_last.pdb",
        f"output/{prefix}_nvt_last.chk",
        f"output/{prefix}_npt_traj.csv",
        f"output/{prefix}_npt_traj.dcd",
        f"output/{prefix}_npt_last.pdb",
        f"output/{prefix}_npt_last.chk",
        f"output/{prefix}_nve_traj.csv",
        f"output/{prefix}_nve_traj.dcd",
        f"output/{prefix}_nve_last.pdb",
        f"output/{prefix}_nve_last.chk",
    ],
)

## Initial state

In [ ]:
# Build initial state (1728 = (6*2)**3 ions)
# Same settings as in Wang 2020 (https://doi.org/10.1063/5.0023225)
system, topology, atnums, atcoords_init = build_nacl_lattice(6, density)
add_nacl_forces(system, topology, do_charge=True, cutoff=1.5 * unit.nanometer)

with open(f"output/{prefix}_first.pdb", "w") as f:
    PDBFile.writeFile(topology, atcoords_init, f)
# Visualize the initial geometry.
view = nglview.show_mdtraj(mdtraj.load(f"output/{prefix}_first.pdb"))
view.add_unitcell()
view

## First Equilibration in the NVT ensemble

In [ ]:
runmd(
    f"{prefix}_nvt",
    system,
    topology,
    nstep=nstep_nvt,
    timestep=timestep,
    stride=stride,
    atcoords=atcoords_init,
    temperature=temperature,
    seed=seed,
)

In [ ]:
traj = mdtraj.load(f"output/{prefix}_nvt_traj.dcd", top=f"output/{prefix}_first.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots(f"{prefix}_nvt")

## Second Equilibration in the NpT ensemble

In [ ]:
runmd(
    f"{prefix}_npt",
    system,
    topology,
    nstep=nstep_npt,
    timestep=timestep,
    stride=stride,
    load_checkpoint_from=f"output/{prefix}_nvt_last.chk",
    temperature=temperature,
    pressure=pressure,
    seed=seed + 12345,
)

In [ ]:
traj = mdtraj.load(f"output/{prefix}_npt_traj.dcd", top=f"output/{prefix}_first.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots(f"{prefix}_npt")

## Initial Production in the NVE ensemble

In [ ]:
runmd(
    f"{prefix}_nve",
    system,
    topology,
    nstep=nstep_nve,
    timestep=timestep,
    stride=stride,
    load_checkpoint_from=f"output/{prefix}_npt_last.chk",
    reset_stepcounter=True,
)

In [ ]:
traj = mdtraj.load(f"output/{prefix}_nve_traj.dcd", top=f"output/{prefix}_first.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots(f"{prefix}_nve")